# Descritores de Imagens via Histograma de Cor

Exemplos baseados em https://github.com/xn2333/OpenCV/blob/master/Seminar_Image_Processing_in_Python.ipynb*







# Instalando Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv 
from google.colab.patches import cv2_imshow # for image display
from skimage import io
from PIL import Image 
import matplotlib.pylab as plt

# Vamos construir nosso dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_zero():
  import os

  path = '/content/drive/MyDrive/hourlyloonacatcher/'

  dataset = {"group": [], "filename": [], "image": [], "post": []}
  for filename in os.listdir(path + 'loona/'):
      img = cv.imread(path + 'loona/' + filename)
      if img is not None:
          dataset["group"].append('loona')
          dataset["filename"].append(filename)
          dataset["image"].append(img)
          dataset["post"].append(False)

  for filename in os.listdir(path + 'deukae/'):
      img = cv.imread(path + 'deukae/' + filename)
      if img is not None:
          dataset["group"].append('deukae')
          dataset["filename"].append(filename)
          dataset["image"].append(img)
          dataset["post"].append(False)

  return dataset

# Extraindo características do dataset usando Histograma de Cor

In [ ]:
def prepareX(dataset):
  color = ('b','g','r')

  dataset_hist_r = []
  dataset_hist_g = []
  dataset_hist_b = []

  counter = 0
  for image in dataset["image"]:
    hists = {}
    for i,col in enumerate(color):
      histr = cv.calcHist([image],[i],None,[256],[0,256])
      if col == 'r': dataset_hist_r.append(histr)
      if col == 'g': dataset_hist_g.append(histr)
      if col == 'b': dataset_hist_b.append(histr)


  X_r = np.array(dataset_hist_r)
  length = np.sqrt((X_r**2).sum(axis=1))[:,None]
  X_r = X_r / length

  X_g = np.array(dataset_hist_g)
  length = np.sqrt((X_g**2).sum(axis=1))[:,None]
  X_g = X_g / length


  X_b = np.array(dataset_hist_b)
  length = np.sqrt((X_b**2).sum(axis=1))[:,None]
  X_b = X_b / length

  X = np.concatenate((X_r,X_g,X_g),axis=1)
  X.shape

  X = X.reshape(X.shape[0],X.shape[1])
  X.shape

  return X

# Agrupamento de Imagens

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

def do_kmeans(X, n_clusters=100):
  kmeans = KMeans(n_clusters=n_clusters).fit(X)
  return kmeans

## Escolher a imagem

In [ ]:
def get_probs(loona_prob = [1.0/12] * 12, deukae_prob = [1.0/7] * 7):
  loona = [
      'heejin',
      'hyunjin',
      'haseul',
      'yeojin',
      'vivi',
      'kimlip',
      'jinsoul',
      'choerry',
      'yves',
      'chuu',
      'gowon',
      'oliviahye'
  ]

  deukae = [
      "jiu",
      "sua",
      "siyeon",
      "handong",
      "yoohyeon",
      "dami",
      "gahyeon"
  ]

  return loona, loona_prob, deukae, deukae_prob

In [ ]:
def update_probs(loona_prob, deukae_prob, loona, deukae, loona_choice, deukae_choice):
  for i in range(len(loona)):
    if loona[i] != loona_choice:
      loona_prob[i] += 1.0/12
    else:
      loona_prob[i] = 1.0/12

  for i in range(len(deukae)):
    if deukae[i] != deukae_choice:
      deukae_prob[i] += 1.0/7
    else:
      deukae_prob[i] = 1.0/7

  return loona_prob, deukae_prob

In [ ]:
import random 

n_clusters = 50

def first_run():  

  dataset = load_zero()
  X = prepareX(dataset)
  kmeans = do_kmeans(X, n_clusters) 

  imageLoona = None
  imageDeukae = None

  face_flag = 0

  images_to_post = {'loona': [], 'deukae': []}

  # first time

  loona, loona_prob, deukae, deukae_prob = get_probs()
  loona_choice = random.choice(loona, weights=loona_prob) 
  deukae_choice = random.choice(deukae, weights=deukae_prob) 
  loona_prob, deukae_prob = update_probs(loona_prob, deukae_prob, loona, deukae, loona_choice, deukae_choice)

  for i in range(192):
    print(loona_choice, deukae_choice) 
    imageLoona, imageDeukae = -1, -1
    clusterList = random.choices(range(n_clusters), k=n_clusters)
    #get image pair for post
    for c in clusterList:
      for image_id, cluster in enumerate(kmeans.labels_):
          if not dataset["post"][image_id]:
            if cluster == c:
              if loonaList[0] in dataset["filename"][image_id]:
                print(image_id)
                imageLoona = image_id
                break

      for image_id, cluster in enumerate(kmeans.labels_):
          if not dataset["post"][image_id]:
            if cluster == c:
              if deukaeList[0] in dataset["filename"][image_id]:
                print(image_id)
                imageDeukae = image_id
                break

      if (imageLoona != -1) and (imageDeukae != -1):
        dataset["post"][imageLoona] = True
        dataset["post"][imageDeukae] = True
        # new pair
        loona_choice = random.choice(loona, weights=loona_prob) 
        deukae_choice = random.choice(deukae, weights=deukae_prob) 
        loona_prob, deukae_prob = update_probs(loona_prob, deukae_prob, loona, deukae, loona_choice, deukae_choice)
        # get out and choose right image
        break
    
    if (imageLoona != -1) and (imageDeukae != -1):
      images_to_post['loona'].append(dataset["filename"][imageLoona])
      images_to_post['deukae'].append(dataset["filename"][imageDeukae])
    else:
      print("No more image combinations for the pair")

    #cv2_imshow(dataset["image"][imageLoona])
    #cv2_imshow(dataset["image"][imageDeukae])

  # generate dfs for saving

  del dataset['image']
  df_dataset = pd.DataFrame(dataset)

  df_images_to_post = pd.DataFrame(images_to_post)

  df_loona = {'name': [], 'prob': []}
  df_loona['name'] = loona
  df_loona['prob'] = loona_prob
  df_loona = pd.DataFrame(df_loona)

  df_deukae = {'name': [], 'prob': []}
  df_deukae['name'] = deukae
  df_deukae['prob'] = deukae_prob
  df_deukae = pd.DataFrame(df_deukae)

  df_dataset.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_dataset.csv')
  df_images_to_post.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/images_to_post.csv')
  df_loona.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_loona.csv')
  df_deukae.to_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_deukae.csv')

In [ ]:
# when no images pairs were generated previously
first_run()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/hourlyloonacatcher_dfs/df_dataset.csv').drop(columns='Unnamed: 0').reset_index().drop(columns='index')

In [ ]:
len(df[df['post'] == True])